# Notebook para geração do módulo de Dataset usando Pytorch Lightning

O objetivo deste notebook é gerar um módulo de dataset com as seguintes funções:

- Separe o texto em treino, teste e validação.
- Gere os labels automaticamente a partir dos textos de dataset em `data/IWSLT/raw`.

In [7]:
import pytorch_lightning as pl
import os
from pathlib import Path

dataset_path = Path('../../data/IWSLT/raw/')

In [2]:
class IWSLTDataset(pl.LightningDataModule):
    def __init__(self, dataset_path, tokenizer, ref=True):
        super().__init__()
        self.dataset_path = dataset_path
    
    def prepare_data(self):
        # called on 1 gpu
        pass

    def setup(self):
        # called on every GPU
        pass
        
    def train_dataloader(self):
        pass

    def val_dataloader(self):
        pass

    def test_dataloader(self):
        pass

In [12]:
import transformers
x = os.walk(dataset_path)
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

filepath = os.path.join(dataset_path, 'train.txt')

In [20]:
with open(filepath) as f:
    data = f.read()

In [22]:
data.split()

['it',
 'can',
 'be',
 'a',
 'very',
 'complicated',
 'thing',
 ',COMMA',
 'the',
 'ocean',
 '.PERIOD',
 'and',
 'it',
 'can',
 'be',
 'a',
 'very',
 'complicated',
 'thing',
 ',COMMA',
 'what',
 'human',
 'health',
 'is',
 '.PERIOD',
 'and',
 'bringing',
 'those',
 'two',
 'together',
 'might',
 'seem',
 'a',
 'very',
 'daunting',
 'task',
 '.PERIOD',
 'but',
 'what',
 'i',
 "'m",
 'going',
 'to',
 'try',
 'to',
 'say',
 'is',
 'that',
 'even',
 'in',
 'that',
 'complexity',
 ',COMMA',
 'there',
 "'s",
 'some',
 'simple',
 'themes',
 'that',
 'i',
 'think',
 ',COMMA',
 'if',
 'we',
 'understand',
 ',COMMA',
 'we',
 'can',
 'really',
 'move',
 'forward',
 '.PERIOD',
 'and',
 'those',
 'simple',
 'themes',
 'are',
 "n't",
 'really',
 'themes',
 'about',
 'the',
 'complex',
 'science',
 'of',
 'what',
 "'s",
 'going',
 'on',
 ',COMMA',
 'but',
 'things',
 'that',
 'we',
 'all',
 'pretty',
 'well',
 'know',
 '.PERIOD',
 'and',
 'i',
 "'m",
 'going',
 'to',
 'start',
 'with',
 'this',
 'on